# Private Predictions with TFE Keras


## Step 3: Private Prediction using TFE Keras - Serving (Client)

Congratulations! After training your model with normal Keras and securing it with TFE Keras, you are ready to request some private predictions. 

In [1]:
import numpy as np
import tensorflow as tf
import tf_encrypted as tfe

from tensorflow.keras.datasets import mnist

/anaconda3/envs/py35_int64/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Data

Here, we preprocess our MNIST data. This is identical to how we preprocessed during training.

In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Set up `tfe.serving.QueueClient`

Before querying the model, you just have to connect to it. To do so, you can create a client with `tfe.serving.QueueClient`. This creates a TFE queueing server on the client side that connects to the queueing server set up by `tfe.serving.QueueServer` in **b - Secure Model Serving**. The queue will be responsible for secret sharing the plaintext data before submitting the shares in a prediction request.

Note that we have to use the same configuration as used by the server, including player configuration and protocol.

In [3]:
config = tfe.RemoteConfig.load("/tmp/tfe.config")

tfe.set_config(config)
tfe.set_protocol(tfe.protocol.SecureNN())

In [4]:
input_shape = (1, 28, 28, 1)
output_shape = (1, 10)

In [5]:
client = tfe.serving.QueueClient(
    input_shape=input_shape,
    output_shape=output_shape)

In [6]:
sess = tfe.Session(config=config)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



## Query Model

You are ready to get some private predictions! Calling `client.run` will insert the `image` into the queue created above, secret share the data locally, and submit the shares to the model server in **b - Secure Model Serving**.

In [7]:
# User inputs
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [8]:
for image, expected_label in zip(images, expected_labels):
    
    res = client.run(
        sess,
        image.reshape(1, 28, 28, 1))
    
    predicted_label = np.argmax(res)
    
    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "incorrectly",
        predicted_label))

The image had label 7 and was correctly classified as 7
The image had label 2 and was correctly classified as 2
The image had label 1 and was correctly classified as 1


This is great. You are able to classify these three images correctly! But what's special about these predictions is that you haven't revealed any private information to get this service. The model host never saw your input data or your predictions, and you never downloaded the model. You were able to get private predictions on encrypted data with an encrypted model!

Before we rush off to apply this in our own apps, let's quickly go back to **b - Secure Model Serving** to clean up our served model.